Cryptocurrency trading bot.

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_output.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for arbitrary intervals.


# Library imports.
from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.renko import get_renko_trigger
from os import mkdir
from os.path import exists, join
from sys import float_info as sflt
from numpy import log
from pandas_ta.utils._core import signed_series, recent_minimum_index

import datetime
import pandas_ta as ta
import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_output(Crypto_logger_base):
    def __init__(self, delay=6, interval_input='15s', interval='15s', buffer_size=100, 
                 input_log_name='input', second_screener=False):
        """
        :param delay: delay between Binance API requests. Minimum calculated was 5 seconds.
        :param interval_input: OHLCV interval from input log. Default is 15 seconds.
        :param interval: OHLCV interval to log. Default is 15 seconds.
        :param buffer_size: buffer size to avoid crashing on low memory.
        :param directory: the directory where to output the logs.
        """
        self.data_before = pd.DataFrame()
        input_log_name = 'crypto_' + input_log_name + '_log_'
        self.load_from_ohlcv = interval_input != interval
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_output_log_' + interval, 
                         second_screener=second_screener, raw=False, precise=True)

        #if not self.load_from_ohlcv:
        #    self.input_log_screened_up_name = \
        #        join(self.directory, input_log_name + interval_input + '_screened_up.txt')

        self.input_log_name = \
            join(self.directory, input_log_name + interval_input + '.txt')
        self.input_log_screened_name = \
            join(self.directory, input_log_name + interval_input + '_screened.txt')

    def get_screened(self, data_after, price_threshold=5.0, volume_threshold=300.0):
        price_movers = pd.DataFrame()
        volume_movers = pd.DataFrame()
        data_before = self.data_before
        if data_before.size != 0:
            data_before.columns = data_before.columns.swaplevel(0, 1)
            data_after.columns = data_after.columns.swaplevel(0, 1)
            price_before = data_before['close'].pct_change(1)
            price_after = data_after['close'].pct_change(1)
            volume_before = data_before['volume'].shift(1)
            volume_after = data_after['volume'].shift(1)
            price_percent_change_before = \
                ((data_before['close'].pct_change(1) - price_before) / price_before)
            price_percent_change_after = \
                ((data_after['close'].pct_change(1) - price_after) / price_after)
            volume_percent_change_before = ((data_before['volume'] - volume_before) / volume_before)
            volume_percent_change_after = ((data_after['volume'] - volume_after) / volume_after)
            price_movers = \
                ((price_percent_change_after - price_percent_change_before) * 100) > price_threshold
            volume_movers = \
                ((volume_percent_change_after - volume_percent_change_before) * 100) > volume_threshold
            price_movers = data_after[price_movers].columns.tolist()
            volume_movers = data_after[volume_movers].columns.tolist()
            data_before.columns = data_before.columns.swaplevel(0, 1)
            data_after.columns = data_after.columns.swaplevel(0, 1)
        self.data_before = data_after
        return price_movers + volume_movers

    def screen(self, dataset):
        def filter_in_market(function, dataset):
            def f(x):
                x = x.loc[:,~x.columns.duplicated()]
                return function(x)
            tickers_list = dataset.columns.get_level_values(0).unique().tolist()
            return pd.Series([ticker for ticker in tickers_list if f(dataset[ticker])], dtype='str')

        def get_relative_volume_levels_smoothed_thresholded(data):
            try:
                volume = data['volume']
                #volume = volume.groupby(pd.Grouper(freq='D')).cumsum()
                volume = volume.groupby(pd.Grouper(freq='24h')).cumsum()
                #volume = volume.groupby(pd.Grouper(freq='60m')).cumsum()
                rvol = (volume / volume.shift(1))
                rvol = rvol.fillna(method='pad')
                bar_up = (ticker['close'] > ticker['open'])
                bar_up |= (ticker['close'] == ticker['open']) & (ticker['close'].diff() > 0)
                bar_up = bar_up.astype(int)
                bar_up = bar_up * 2 - 1
                rvol *= bar_up
                rvol_indicator = ta.hma(rvol, length=14, talib=True)
                rvol_indicator = rvol_indicator.rename('relative_volume_levels_smoothed')
                #threshold = (ta.sma(rvol, length=100, talib=True) + ta.stdev(rvol, length=100, talib=True))
                threshold = 2
                rvol_thresholded = (rvol_indicator > threshold).iloc[-1]
            except:
                rvol_thresholded = False
            return rvol_thresholded

        def get_not_square_wave_trigger_1(data):
            return not (data.iloc[-4:]['close'].unique().size < 2)

        def get_not_square_wave_trigger_2(data):
            return not (data.iloc[-15:]['close'].unique().size < 6)

        def get_not_square_wave_trigger_3(data):
            return (data[['open', 'high', 'low', 'close']].nunique(axis='columns') > 2).tail(2).all()

        def get_bullish_price_trigger(data):
            return (data['close'] > data['high'].shift(1)).iloc[-1]

        def get_positive_RSI_trigger(data):
            RSI_6 = data.ta.rsi(length=6, talib=True)
            RSI_12 = data.ta.rsi(length=12, talib=True)
            RSI_24 = data.ta.rsi(length=24, talib=True)
            data = ((RSI_6 > RSI_12) | (RSI_6 > RSI_24) | (RSI_12 > RSI_24))
            return data.iloc[-1]

        def get_positive_momentum_trigger(data):
            KDJ = data.ta.kdj(length=5, signal=3, talib=True)
            return ((KDJ['J_5_3'] > KDJ['D_5_3']) & (KDJ['J_5_3'] > KDJ['K_5_3'])).iloc[-1]

        def get_positive_JMA_trigger(data):
            JMA = data.ta.jma(length=7, phase=0, talib=True)
            return (data['close'] < JMA).iloc[-1]

        def get_ease_of_movement(data):
            eom = ((data['high'] - data['low']) / (2 * data['volume'] + 1))
            eom *= (data['high'].diff(1) + data['low'].diff(1))
            precision = eom.abs().max()
            if precision < 1:
                eom *= 1 / precision
            return eom

        def get_ease_of_movement_trigger(data):
            data[['open', 'high', 'low', 'close']] += sflt.epsilon
            data[['volume']] += 1

            log_price = log(data['close'])
            price_trough_index = recent_minimum_index(signed_series(log_price, initial=None))
            price_slope = ta.slope(close=log_price, length=price_trough_index, as_angle=True, 
                                   to_degrees=True, talib=True)

            EOM = get_ease_of_movement(data)
            EOM_trough_index = recent_minimum_index(signed_series(EOM, initial=None))
            EOM_slope = ta.slope(close=EOM, length=EOM_trough_index, as_angle=True, 
                                 to_degrees=True, talib=True)

            trigger = (price_slope <= EOM_slope)
            #trigger &= (EOM_slope >= 0.25)
            trigger &= (EOM_slope > 0.0)

            #breakout_trigger = ((EOM.shift(1) < 0.0) & (EOM > 0.0))
            #trigger |= breakout_trigger
            return trigger.iloc[-1]

        '''
        def get_positive_PVR_trigger(data):
            price = data['close']
            volume = data['volume']
            volume.iloc[-1] *= volume_multiplier
            price_trigger = (price.diff() > 0)
            volume_trigger = (volume.diff() > 0)
            trigger = (price_trigger & volume_trigger)
            return trigger.iloc[-1]
        '''

        def get_rising_volume_trigger(data):
            return (data['rolling_base_volume'].diff(1) > 0).iloc[-1]

        def get_RSI_reversal_trigger(data, rsi_length=2, upper_threshold=95, 
                                     lower_threshold=5, positive=True):
            RSI = data.ta.rsi(length=rsi_length, talib=True)
            RSI_prev = RSI.shift(1)
            thresholds_bear = -((RSI_prev >= upper_threshold) & (RSI < upper_threshold)).astype(int)
            thresholds_bull = ((RSI_prev <= lower_threshold) & (RSI > lower_threshold)).astype(int)
            thresholds = (thresholds_bear + thresholds_bull)
            thresholds = thresholds.replace(to_replace=0, method='pad')
            return (thresholds == (1 if positive else -1)).iloc[-1]

        def get_heikin_ashi_trigger(data):
            def get_positive_trend_strength_trigger(data):
                ADX = data.ta.adx(talib=True)
                return (ADX['ADX_14'] < 0.20).iloc[-3] and (ADX['ADX_14'] > 0.20).iloc[-2]

            def get_not_negative_trend_strength_trigger(data):
                ADX = data.ta.adx(length=14, lensig=8, talib=True)
                return ((ADX['DMP_14'] > ADX['DMN_14']) and (ADX['ADX_14'] > 0.30)).iloc[-1]

            def get_not_negative_rebound_trigger(data):
                CCI = data.ta.cci(length=22, talib=True)
                MFI = data.ta.mfi(length=11, talib=True)
                return ((CCI > 0) or (MFI > 20)).iloc[-1]

            def get_positive_choppiness_trigger(data):
                CHOP = data.ta.chop(talib=True)
                return CHOP.iloc[-1] < 38.2

            def get_positive_phase_trigger(data):
                MACD = data.ta.macd(talib=True)
                histogram = MACD['MACDs_12_26_9'] - MACD['MACD_12_26_9']
                return ((histogram.iloc[-1] > histogram.iloc[-2]) or \
                        (MACD['MACD_12_26_9'].iloc[-1] > MACD['MACDs_12_26_9'].iloc[-1]))

            def get_positive_RSI_trigger(data):
                RSI_5 = data.ta.rsi(length=5, talib=True)
                return ((RSI_5 >= 60) & (RSI_5 <= 65)).iloc[-1]

            def get_negative_PVR_trigger(data):
                price_trigger = (data['close'].iloc[-1] < data['close'].iloc[-2])
                volume_trigger = (data['volume'].iloc[-1] > data['volume'].iloc[-2])
                return price_trigger and volume_trigger

            def get_buy_trigger(data):
                return get_not_negative_rebound_trigger(data) and \
                        (get_positive_choppiness_trigger(data) or \
                        get_positive_trend_strength_trigger(data))

            def get_sell_trigger(data):
                return (((not get_positive_choppiness_trigger(data)) or \
                         get_negative_trend_strength_trigger(data) or \
                         (not get_positive_phase_trigger(data))) or \
                        get_not_negative_rebound_trigger(data))

            heikin_ashi = data.ta.ha(talib=True)
            heikin_ashi_dataset_1 = heikin_ashi.rename(columns={'HA_open': 'open', 
                                                                'HA_high': 'high', 
                                                                'HA_low': 'low', 
                                                                'HA_close': 'close'})
            #heikin_ashi = heikin_ashi_dataset_1.ta.ha(talib=True)
            #heikin_ashi_dataset_2 = heikin_ashi.rename(columns={'HA_open': 'open', 
            #                                                    'HA_high': 'high', 
            #                                                    'HA_low': 'low', 
            #                                                    'HA_close': 'close'})
            #heikin_ashi = heikin_ashi_dataset_2.ta.ha(talib=True)
            #heikin_ashi_dataset_3 = heikin_ashi.rename(columns={'HA_open': 'open', 
            #                                                    'HA_high': 'high', 
            #                                                    'HA_low': 'low', 
            #                                                    'HA_close': 'close'})
            #if get_not_negative_rebound_trigger(heikin_ashi_dataset_1):
            #    return True
            return True \
                if get_positive_phase_trigger(heikin_ashi_dataset_1) \
                else (get_not_negative_rebound_trigger(heikin_ashi_dataset_1) or \
                      get_not_negative_trend_strength_trigger(heikin_ashi_dataset_1))

        def screen_one(pair):
            frequency = pd.tseries.frequencies.to_offset((pair.index[1:] - pair.index[:-1]).min())
            frequency_1min = pd.tseries.frequencies.to_offset('1min')
            frequency_30min = pd.tseries.frequencies.to_offset('30min')
            frequency_1h = pd.tseries.frequencies.to_offset('1h')
            if frequency < frequency_1min:
                pair['volume'] = pair['rolling_base_volume'].copy()
            else:
                pair['volume'] = pair['base_volume'].copy()
            if frequency == frequency_30min:
                if get_not_square_wave_trigger_1(pair):
                    if get_not_square_wave_trigger_2(pair):
                        #if get_bullish_price_trigger(pair):
                        #if get_heikin_ashi_trigger(pair):
                        if get_renko_trigger(pair, compress=False, 
                                             direction_type='long', 
                                             trigger_type='simple', 
                                             method='atr', plot=False):
                            return True
            elif frequency == frequency_1h:
                if get_relative_volume_levels_smoothed_thresholded(pair):
                    return True
            else:
                if get_not_square_wave_trigger_1(pair):
                    if get_not_square_wave_trigger_2(pair):
                        if frequency < frequency_1min:
                            return True 
                        else:
                            if get_rising_volume_trigger(pair):
                                if get_heikin_ashi_trigger(pair):
                                    return True
            return False

        #if not self.load_from_ohlcv:
        #    if exists(self.input_log_screened_up_name):
        #        input_filtered_up = pd.read_csv(self.input_log_screened_up_name, header=0, index_col=None)
        if exists(self.input_log_screened_name):
            input_filtered = pd.read_csv(self.input_log_screened_name, header=0, index_col=0)
            input_filter = set(input_filtered['symbol'].tolist())
            #if not self.load_from_ohlcv:
            #    input_filter = input_filter & set(input_filtered_up['symbol'].tolist())
            old_columns = set(dataset.columns.get_level_values(0).tolist())
            new_columns = list(input_filter & old_columns)
            dataset = dataset[new_columns]

            #assets = self.get_screened(dataset, price_threshold=1.0, volume_threshold=1.0)
            #input_filtered_movers = input_filtered[input_filtered['symbol'].isin(assets)]
            #input_filtered_movers.to_csv(self.input_log_screened_name, mode='a')
            assets = filter_in_market(screen_one, dataset)
            return input_filtered[input_filtered['symbol'].isin(assets)]
        else:
            return None

    def screen_(self, dataset):
        return dataset

    def resample_from_raw(self, df):
        df = df[['symbol', 'lastPrice', 'volume', 'quoteVolume']]
        df = df.rename(columns={'lastPrice': 'close', 
                                'volume': 'rolling_base_volume', 
                                'quoteVolume': 'rolling_quote_volume'})
        df = df.pivot_table(index=['date'], columns=['symbol'], 
                            values=['close', 'rolling_base_volume', 
                                    'rolling_quote_volume'], 
                            aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                     'rolling_base_volume': 'max', 
                                     'rolling_quote_volume': 'max'})
        df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                               names=(None, 'symbol'))
        df = df.rename(columns={'close_first': 'open', 
                                'close_max': 'high', 
                                'close_min': 'low', 
                                'close_last': 'close', 
                                'rolling_base_volume_max': 'rolling_base_volume', 
                                'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                       level=0)
        df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
        df['rolling_base_volume'].iloc[0] = 0
        df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
        df['rolling_quote_volume'].iloc[0] = 0
        df = df.sort_index().iloc[1:]
        df.columns = df.columns.swaplevel(0, 1)
        return df

    def get(self):
        if self.load_from_ohlcv:
            dataset = pd.read_csv(self.input_log_name, header=[0, 1], index_col=0)
        else:
            dataset = pd.read_csv(self.input_log_name, header=0, index_col=0)
            dataset = self.resample_from_raw(dataset)
        return dataset.sort_index().tail(2)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        cryptocurrency/crypto_logger_input.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger circular buffered for N time precision.

# Library imports.
from cryptocurrency.crypto_logger_base import Crypto_logger_base
from cryptocurrency.authentication import Cryptocurrency_authenticator
from cryptocurrency.exchange import Cryptocurrency_exchange
from cryptocurrency.conversion import get_base_asset_from_pair, get_quote_asset_from_pair
from os import mkdir
from os.path import exists, join

import datetime
import pandas as pd
pd.options.mode.chained_assignment = None

class Crypto_logger_input(Crypto_logger_base):
    def __init__(self, client=None, delay=4.7, interval='1min', buffer_size=20000, 
                 price_percent=5.0, volume_percent=1.0):
        """
        :param interval: OHLCV interval to log. Default is 1 minute.
        :param delay: delay between Binance API requests. Minimum calculated was 5 seconds.
        :param buffer_size: buffer size to avoid crashing on low memory.
        :param directory: the directory where to output the logs.
        """
        self.resample = None
        self.price_percent = price_percent
        self.volume_percent = volume_percent
        super().__init__(interval=interval, delay=delay, buffer_size=buffer_size, 
                         directory='crypto_logs', log_name='crypto_input_log_' + interval, 
                         second_screener=False, raw=True, precise=False)

        #self.log_screened_up_name = self.log_name.replace('.txt', '') + '_screened_up.txt'

        authenticator = Cryptocurrency_authenticator(use_keys=False, testnet=False)
        self.client = authenticator.spot_client

        exchange = Cryptocurrency_exchange(client=self.client, directory=self.directory)
        self.exchange_info = exchange.info

    def filter_movers(self, dataset, count=1000, price_percent=5.0, volume_percent=1.0):
        dataset = dataset.reset_index()
        dataset[['priceChangePercent', 'rolling_quote_volume']] = \
            dataset[['priceChangePercent', 'rolling_quote_volume']].astype(float)
        dataset['last_price_move'] = dataset['priceChangePercent'].copy()
        dataset['last_volume_move'] = dataset['rolling_quote_volume'].copy()
        movers = dataset.groupby(['symbol'])
        dataset = dataset.drop(columns=['last_price_move', 'last_volume_move'])
        price_movers = movers['last_price_move']
        volume_movers = movers['last_volume_move']
        price_movers = price_movers.agg(lambda x: x.diff(1).abs().iloc[-1])
        volume_movers = volume_movers.agg(lambda x: (100 * x.pct_change(1)).iloc[-1])
        price_movers = price_movers.sort_values(ascending=False)
        volume_movers = volume_movers.sort_values(ascending=False)
        price_movers = price_movers[price_movers > 0.0]
        #price_movers_int = price_movers.copy().dropna().astype(int)
        #price_movers_int = price_movers_int[price_movers_int >= 0]
        price_movers = price_movers.to_frame(name='last_price_move')
        volume_movers = volume_movers.to_frame(name='last_volume_move')
        #price_movers_int.to_csv(self.log_screened_up_name)
        movers = pd.concat([price_movers, volume_movers], axis='columns')
        movers = movers.reset_index()
        price_movers_mask = movers['last_price_move'] > price_percent
        volume_movers_mask = movers['last_volume_move'] > volume_percent
        movers = movers[price_movers_mask & volume_movers_mask]
        movers = movers.sort_values(by=['last_volume_move', 'last_price_move'], ascending=False)
        movers = movers.tail(count).reset_index(drop=True)
        return dataset.merge(right=movers, how='right', on=['symbol']).set_index('date')

    def get_highest_daily_volume_pair_associated_with_base_asset(self, base_asset):
        pairs = self.exchange_info[self.exchange_info['baseAsset'] == base_asset]['symbol'].tolist()
        filtered_pairs = self.conversion_table[self.conversion_table['symbol'].isin(pairs)]
        filtered_pairs = filtered_pairs.sort_values(by=['rolling_quote_volume'], ascending=False)
        try:
            filtered_pairs = filtered_pairs['symbol'].iloc[0]
        except IndexError:
            filtered_pairs = None
        return filtered_pairs

    def screen(self, dataset):
        dataset = dataset[['symbol', 'lastPrice', 'priceChangePercent', 
                           'bidPrice', 'askPrice', 'bidQty', 'askQty', 
                           'volume', 'quoteVolume', 'count']]
        dataset[['priceChangePercent', 'lastPrice', 'bidPrice', 'askPrice', 
                 'bidQty', 'askQty', 'volume', 'quoteVolume', 'count']] = \
            dataset[['priceChangePercent', 'lastPrice', 'bidPrice', 'askPrice', 
                     'bidQty', 'askQty', 'volume', 'quoteVolume', 'count']].astype(float)
        dataset = dataset.rename(columns={'volume': 'rolling_base_volume', 
                                          'quoteVolume': 'rolling_quote_volume'})
        #dataset = dataset[dataset['quote_volume'] > 50000]
        dataset['bidAskChangePercent'] = \
            ((dataset['askPrice'] - dataset['bidPrice']) / dataset['askPrice'])
        dataset['bidAskQtyPercent'] = \
            (dataset['bidQty'] / (dataset['bidQty'] + dataset['askQty']))
        dataset[['bidAskChangePercent', 'bidAskQtyPercent']] *= 100
        dataset = dataset.dropna()
        dataset = dataset[dataset['bidAskChangePercent'] < 0.8]
        #dataset = dataset[dataset['bidAskQtyPercent'] > 100.0]
        dataset = self.filter_movers(dataset, count=1000, price_percent=self.price_percent, 
                                     volume_percent=self.volume_percent)
        dataset = dataset.drop_duplicates(subset=['symbol', 'count'], keep='last')
        dataset['base_asset'] = dataset['symbol'].apply(lambda x: get_base_asset_from_pair(x, exchange_info=self.exchange_info))
        dataset['quote_asset'] = dataset['symbol'].apply(lambda x: get_quote_asset_from_pair(x, exchange_info=self.exchange_info))
        #dataset['symbol_2'] = dataset['base_asset'].apply(lambda x: self.get_highest_daily_volume_pair_associated_with_base_asset(base_asset=x))
        #dataset[dataset['symbol_2'] == None]['symbol_2'] = dataset[dataset['symbol_2'] == None]['symbol']
        #dataset['symbol'] = dataset['symbol_2'].copy()
        #dataset = dataset.drop(columns=['symbol_2'])
        return dataset

    def prepare_downsampling(self, dataset):
        dataset['closeTime'] /= 1000
        dataset['openTime'] /= 1000
        dataset['openTime'] = \
            dataset['openTime'].apply(datetime.datetime.fromtimestamp)
        dataset['closeTime'] = \
            dataset['closeTime'].apply(datetime.datetime.fromtimestamp)
        dataset['date'] = pd.DatetimeIndex(dataset['closeTime']).round(self.interval)
        return dataset.set_index('date').sort_index()

    def get(self):
        """Get all pairs data from Binance API."""
        dataset = pd.DataFrame(self.client.get_ticker())
        self.conversion_table = dataset.copy()
        self.conversion_table.to_csv(self.log_name.replace('.txt', '') + '_conversion_table.txt')
        return self.prepare_downsampling(self.conversion_table)

In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

# File:        crypto_logger_input_1min.py
# By:          Samuel Duclos
# For          Myself
# Description: Simple Binance logger output for the 1 minute interval.

# Library imports.
#from cryptocurrency.crypto_logger_input import Crypto_logger_input

crypto_logger_input_1min = Crypto_logger_input(delay=12, interval='1min', buffer_size=20000, 
                                               price_percent=1.0, volume_percent=1.0)
crypto_logger_input_1min.start(append=True, roll=60)

In [ ]:
import pandas as pd

crypto_input_log_1min = 'crypto_logs/crypto_input_log_1min.txt'
df = pd.read_csv(crypto_input_log_1min, header=0, index_col=0)
df.index = pd.to_datetime(df.index, utc=True)
df

In [ ]:
import pandas as pd

crypto_input_log_15s = 'crypto_logs/crypto_input_log_15s.txt'
df = pd.read_csv(crypto_input_log_15s, header=0, index_col=0)
df.index = pd.to_datetime(df.index)
df

In [ ]:
df.columns

In [ ]:
df = df[['symbol', 'lastPrice', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty']]
df = df.rename(columns={'lastPrice': 'close'})
df = df.pivot_table(index=['date'], columns=['symbol'], 
                    values=['close', 'volume', 'bidPrice', 'bidQty', 'askPrice', 'askQty'], 
                    aggfunc={'close': ['first', 'max', 'min', 'last'], 
                             'volume': 'last', 
                             'bidPrice': ['first', 'max', 'min', 'last'], 
                             'bidQty': 'sum', 
                             'askPrice': ['first', 'max', 'min', 'last'], 
                             'askQty': 'sum'})
df['volume'] = df['volume'].fillna(method='pad').fillna(0).diff(1)
df['bidQty']['sum'] = df['bidQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df['askQty']['sum'] = df['askQty']['sum'].fillna(method='pad').fillna(0).diff(1)
df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values])
df = df.rename(columns={'close_first': 'open', 
                        'close_max': 'high', 
                        'close_min': 'low', 
                        'close_last': 'close', 
                        'volume_last': 'volume', 
                        'bidPrice_first': 'bid_open', 
                        'bidPrice_max': 'bid_high', 
                        'bidPrice_min': 'bid_low', 
                        'bidPrice_last': 'bid_close', 
                        'bidQty_sum': 'bid_volume', 
                        'askPrice_first': 'ask_open', 
                        'askPrice_max': 'ask_high', 
                        'askPrice_min': 'ask_low', 
                        'askPrice_last': 'ask_close', 
                        'askQty_sum': 'ask_volume'}, 
               level=0)
df

In [ ]:
df.columns

In [ ]:
df2.columns.get_level_values(1).unique()

In [ ]:
def resample_from_raw(df):
    df = df[['symbol', 'lastPrice', 'volume', 'quoteVolume']]
    df = df.rename(columns={'lastPrice': 'close', 
                            'volume': 'rolling_base_volume', 
                            'quoteVolume': 'rolling_quote_volume'})
    df = df.pivot_table(index=['date'], columns=['symbol'], 
                        values=['close', 'rolling_base_volume', 
                                'rolling_quote_volume'], 
                        aggfunc={'close': ['first', 'max', 'min', 'last'], 
                                 'rolling_base_volume': 'max', 
                                 'rolling_quote_volume': 'max'})
    df.columns = pd.MultiIndex.from_tuples([('_'.join(col[:2]), col[2]) for col in df.columns.values], 
                                           names=(None, 'symbol'))
    df = df.rename(columns={'close_first': 'open', 
                            'close_max': 'high', 
                            'close_min': 'low', 
                            'close_last': 'close', 
                            'rolling_base_volume_max': 'rolling_base_volume', 
                            'rolling_quote_volume_max': 'rolling_quote_volume'}, 
                   level=0)
    df['rolling_base_volume'] = df['rolling_base_volume'].fillna(method='pad')
    df['rolling_base_volume'].iloc[0] = 0
    df['rolling_quote_volume'] = df['rolling_quote_volume'].fillna(method='pad')
    df['rolling_quote_volume'].iloc[0] = 0
    df = df.sort_index().iloc[1:]
    df.columns = df.columns.swaplevel(0, 1)
    return df

df2 = resample_from_raw(df)
df2

In [ ]:
df2[(df2['NEBLBUSD']['volume'].diff() < 0)]['NEBLBUSD']['volume']

In [ ]:
#df2.columns = df2.columns.swaplevel(0, 1)
(~((df2[df2['volume'] > 0]['volume']).isna())).any().any()

In [ ]:
df2[(df2['volume'] > 0)]

In [ ]:
#%rm -rf crypto_logs